In [2]:
import sys,os,inspect

sys.path.insert(1, os.path.join(sys.path[0], '..'))

import pandas as pd
import numpy as np
import json
import tqdm
import datetime as dt
from io import StringIO

import azure.cosmos.cosmos_client as azurecosmos
import azure.storage.blob as azureblob

from transformers import AutoModel, AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification
import torch

from cs_config import *
from cs_tools import *

/anaconda/envs/counterspeech/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Remove AC tweets by SU accounts

In [17]:
su = pd.read_csv('../data/mps_seed_info.csv', dtype={'twitter_user_id': object})
su

,os_user_id,name,twitter_screen_name,party,constituency,twitter_followers,twitter_user_id,twitter_verified,twitter_found,token,seed_flag,twitter_screen_name_orig,twitter_screen_name_old,updated
0,1,Aaron Bell,aaronbell4nul,Conservative,Newcastle-under-Lyme,5314.0,240808845,True,True,MP,True,aaronbell4nul,[],NaN
1,2,Abena Oppong-Asare,abenaopp,Labour,Erith and Thamesmead,17255.0,350223904,True,True,MP,True,abenaopp,[],NaN
2,3,Adam Afriyie,adamafriyie,Conservative,Windsor,17412.0,22031058,True,True,MP,True,adamafriyie,[],NaN
3,4,Afzal Khan,afzal4gorton,Labour,"Manchester, Gorton",32676.0,202610289,True,True,MP,True,afzal4gorton,[],NaN
4,5,Alan Brown,alanbrownsnp,Scottish National Party,Kilmarnock and Loudoun,12506.0,3011043981,True,True,MP,True,alanbrownsnp,[],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,587,Yasmin Qureshi,yasminqureshimp,Labour,Bolton South East,24956.0,72341341,True,True,MP,True,yasminqureshimp,[],NaN
587,588,Yvette Cooper,yvettecoopermp,Labour,"Normanton, Pontefract and Castleford",344927.0,328634628,True,True,MP,True,yvettecoopermp,[],NaN
588,589,Yvonne Fovargue,y_fovarguemp,Labour,Makerfield,13811.0,568174146,True,True,MP,True,y_fovarguemp,[],NaN
589,590,Zarah Sultana,zarahsultana,Labour,Coventry South,218758.0,3056307455,True,True,MP,True,zarahsultana,[],NaN


In [37]:
# get tweets that are by MPs

m_cosmos = get_cosmos_client(Cosmos.host, Cosmos.key, Cosmos.mps_db, Cosmos.mps_container)

first = True
array_str = ""
for id in su[su.twitter_user_id.notna()].twitter_user_id.tolist():
    if first: array_str += f"\"{id}\""
    else: array_str += f", \"{id}\""
    first=False

su_twids = query_cosmos_field(
    m_cosmos,
    'id',
    filter=f"c.bucket=\"audience_contact\" and c.valid=true and ARRAY_CONTAINS([{array_str}], c.user_id)"
)

In [38]:
len(su_twids)

(7847, 7847)

In [40]:
# remove tweets with ids found from query
df = pd.read_csv('../data/mps_valid_ac_tweets_idtxt_rc.csv', dtype={'id':object})
df2 = df[~df.id.isin(su_twids)].reset_index(drop=True)
df.shape[0], df2.shape[0], df.shape[0]-df2.shape[0]

(2599492, 2591833, 7659)

In [41]:
df2.to_csv('../data/mps_valid_ac_tweets_idtxt_rc_nosu.csv', index=False, encoding='utf-8-sig')

# Analysis of remaining/replies

In [42]:
df = pd.read_csv('../data/mps_valid_ac_tweets_idtxt_rc_nosu.csv', dtype={'id':object})
df

,id,text_replaced_b,replycount
0,1489750623213948928,"As of 12:00am, 5th February 2022, [MP] owes ou...",0.0
1,1489750748212641794,"Hey [MP] - saw your pathetic ‘drip,drip,drip’ ...",0.0
2,1489750775907692546,Not just in the UK [MP],0.0
3,1489750802570915844,[USER] world leaders are waiting for you on Bi...,0.0
4,1489750860519329805,[USER] would you a factual talking figure chal...,0.0
...,...,...,...
2591828,1520191052979703808,It all started in England and still emanates f...,0.0
2591829,1520191085137612801,[MP] what type of porn was it just out of curi...,1.0
2591830,1520191097993060352,Approximately 90% of #Tigray’s pop. needs huma...,0.0
2591831,1520191151399227393,|'The main hospital in #Ethiopia’s war-ravaged...,0.0


In [12]:
def rc_counts(df, checks=[0,1,2,3,10,100]):
    total = df.shape[0]
    anyrep = df[df.replycount>0].shape[0]

    dfr = pd.DataFrame(data={
        'replies': checks,
        'count': [df[df.replycount==c].shape[0] for c in checks],
        'prop_of_all': [df[df.replycount==c].shape[0]/total for c in checks],
        'prop_of_replies': [df[df.replycount==c].shape[0]/anyrep if c>0 else np.nan for c in checks],
        'at_least_count': [df[df.replycount>=c].shape[0] for c in checks],
        'at_least_prop_of_all': [df[df.replycount>=c].shape[0]/total for c in checks],
        'at_least_prop_of_replies': [df[df.replycount>=c].shape[0]/anyrep if c>0 else np.nan for c in checks],
    })  
    return dfr

In [ ]:
dfr = rc_counts(df)
dfr

In [51]:
df[df.replycount>0].reset_index(drop=True).to_csv('../data/mps_valid_ac_tweets_idtxt_rc_nosu_anyreplies.csv', index=False, encoding='utf-8-sig')

In [50]:
# save copy of df of only tweets with replies - for labelling

df[df.replycount>0].sort_values(by='replycount', ascending=False).iloc[2]

id                                               1499851517859438594
text_replaced_b    We do not mind if the next [USER] TV show is a...
replycount                                                    6498.0
Name: 836179, dtype: object

In [6]:
# upload files to blob storage
csblob = get_blob_client(CounterSpeechBlobStorage.connect_str, 'data')
upload_file_to_blob(csblob, '../data/mps_valid_ac_tweets_idtxt_rc_nosu.csv', 'mps_valid_ac_tweets_idtxt_rc_nosu')
upload_file_to_blob(csblob, '../data/mps_valid_ac_tweets_idtxt_rc_nosu_anyreplies.csv', 'mps_valid_ac_tweets_idtxt_rc_nosu_anyreplies')

In [3]:
dfl = pd.read_csv('../data/mps_valid_ac_tweets_idtxt_rc_nosu_anyreplies_labelled.csv')
dfl

,id,text_replaced_b,replycount,probs,labels
0,1489751225235038211,⁦[MP]⁩ ⁦[MP]⁩ ⁦[MP]⁩ and hundreds of thousands...,1.0,0.609356,1
1,1489751338640683008,Just how deluded is the average #Tory[MP] if t...,2.0,0.668969,1
2,1489751511890640896,It's now just 55 days until the energy price c...,1.0,0.028957,0
3,1489752017824329728,Just a reminder a Labour Peer has been convict...,4.0,0.070316,0
4,1489752568465170432,Hope you've listened to this [MP] . It's one o...,1.0,0.000784,0
...,...,...,...,...,...
243135,1520187619409383424,#iftar dinner with [MP] [USER] organised by Cl...,6.0,0.001530,0
243136,1520188579225473024,Chancellor breaks pledge on energy bills. With...,9.0,0.009216,0
243137,1520189263308103686,[USER] Re: [URL] as U write things suitable 4 ...,1.0,0.005200,0
243138,1520190038209961985,Only females have a cervix [MP] and unfortun...,6.0,0.175117,0


In [4]:
dfl.labels.sum(), dfl.labels.sum()/dfl.shape[0]

(10334, 0.04250226207123468)

In [5]:
dflt = dfl[dfl.labels==1]
dflf = dfl[dfl.labels==0]

In [10]:
dflt.replycount.describe()

count    10334.000000
mean         3.873040
std         32.037394
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max       1722.000000
Name: replycount, dtype: float64

In [9]:
dflf.replycount.describe()

count    232806.000000
mean          8.182328
std          70.466802
min           1.000000
25%           1.000000
50%           1.000000
75%           3.000000
max        8577.000000
Name: replycount, dtype: float64

In [13]:
rc_counts(dflt)

,replies,count,prop_of_all,prop_of_replies,at_least_count,at_least_prop_of_all,at_least_prop_of_replies
0,0,0,0.000000,NaN,10334,1.000000,NaN
1,1,6681,0.646507,0.646507,10334,1.000000,1.000000
2,2,1572,0.152119,0.152119,3653,0.353493,0.353493
3,3,647,0.062609,0.062609,2081,0.201374,0.201374
4,10,47,0.004548,0.004548,530,0.051287,0.051287
5,100,0,0.000000,0.000000,33,0.003193,0.003193


In [14]:
rc_counts(dflf)

,replies,count,prop_of_all,prop_of_replies,at_least_count,at_least_prop_of_all,at_least_prop_of_replies
0,0,0,0.000000,NaN,232806,1.000000,NaN
1,1,139752,0.600294,0.600294,232806,1.000000,1.000000
2,2,34114,0.146534,0.146534,93054,0.399706,0.399706
3,3,14831,0.063705,0.063705,58940,0.253172,0.253172
4,10,1477,0.006344,0.006344,20456,0.087867,0.087867
5,100,25,0.000107,0.000107,2849,0.012238,0.012238


In [15]:
dflt

,id,text_replaced_b,replycount,probs,labels
0,1489751225235038211,⁦[MP]⁩ ⁦[MP]⁩ ⁦[MP]⁩ and hundreds of thousands...,1.0,0.609356,1
1,1489751338640683008,Just how deluded is the average #Tory[MP] if t...,2.0,0.668969,1
5,1489752784018849794,Its ok for [MP] to throw around insults at our...,6.0,0.703037,1
6,1489752912523841537,[MP] . I said to you to watch your back with S...,1.0,0.670325,1
25,1489757089878708228,They try but this isn’t going anywhere and nei...,1.0,0.830350,1
...,...,...,...,...,...
243008,1520147230769954816,It's the same rhetoric you get from [MP] about...,1.0,0.550993,1
243021,1520149183747051520,So [MP] said he opened a link on his phone by ...,3.0,0.932377,1
243040,1520155637728129024,"So [MP] opened up a porn file TWICE, 'by mista...",5.0,0.951278,1
243079,1520165036701982721,.[MP] is a nasty piece of work.,2.0,0.854533,1


In [ ]:
dflt.sort_values(by='probs', ascending=False).head(50)

In [17]:
dflt.replycount.sum()

40024.0